In [1]:
!pip install -q rouge_score evaluate

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64

In [2]:
!wandb login "8a991a809a851bca2196ad726fdc4745d7855fd3"

wandb: WARNING Using legacy-service, which is deprecated. If this is unintentional, you can fix it by ensuring you do not call `wandb.require('legacy-service')` and do not set the WANDB_X_REQUIRE_LEGACY_SERVICE environment variable.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [3]:
import os
import pandas as pd
import torch
import numpy as np
import random
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import (
    prepare_model_for_kbit_training,
    LoraConfig,
    get_peft_model,
    TaskType
)
import wandb
import re
from transformers.integrations import WandbCallback
from transformers.trainer_callback import TrainerCallback


class WandbTableCallback(TrainerCallback):
    """Callback для создания таблицы с примерами в wandb."""

    def __init__(self, trainer, dataset, tokenizer, model, num_fixed_examples=3, num_random_examples=3,
                 log_examples_every=50):
        self.trainer = trainer
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.model = model
        self.num_fixed_examples = num_fixed_examples
        self.num_random_examples = num_random_examples
        self.log_examples_every = log_examples_every

        # Создаем таблицу wandb
        self.examples_data = []  # Храним данные для таблицы
        self.columns = ["Шаг", "Тип", "Оригинал", "Таргет", "Сгенерированное"]

        # Размер датасета
        self.dataset_size = len(dataset)

        # Выбираем фиксированные примеры
        self.fixed_indices = random.sample(range(self.dataset_size), min(num_fixed_examples, self.dataset_size))

    def on_step_end(self, args, state, control, **kwargs):
        """Вызывается после каждого шага обучения."""
        if state.global_step % self.log_examples_every == 0:
            # Безопасное получение значения loss из log_history
            try:
                # Перебираем записи в обратном порядке, чтобы найти последнюю с ключом 'loss'
                current_loss = 0
                if state.log_history:
                    for entry in reversed(state.log_history):
                        if 'loss' in entry:
                            current_loss = entry['loss']
                            break
                wandb.log({"train/loss": current_loss})  # Убран параметр step
            except Exception as e:
                print(f"Ошибка при логировании loss: {e}")
                # Продолжаем выполнение даже при ошибке логирования

            # Создаем таблицу для текущего шага
            current_examples = []

            # 1. Обрабатываем фиксированные примеры
            for idx in self.fixed_indices:
                # Получаем оригинальный текст
                original_text = self._extract_original_text(idx)

                # Получаем целевой текст (таргет)
                target_text = self._extract_target_text(idx)

                # Генерируем текст с помощью модели
                generated_text = self._generate_text(original_text)

                # Сохраняем данные
                current_examples.append(
                    [state.global_step, "Фиксированный", original_text, target_text, generated_text])

            # 2. Выбираем и обрабатываем случайные примеры
            if self.num_random_examples > 0:
                # Выбираем новые случайные индексы (исключая фиксированные)
                available_indices = list(set(range(self.dataset_size)) - set(self.fixed_indices))
                if len(available_indices) > 0:
                    random_indices = random.sample(
                        available_indices,
                        min(self.num_random_examples, len(available_indices))
                    )

                    for idx in random_indices:
                        # Получаем оригинальный текст
                        original_text = self._extract_original_text(idx)

                        # Получаем целевой текст (таргет)
                        target_text = self._extract_target_text(idx)

                        # Генерируем текст с помощью модели
                        generated_text = self._generate_text(original_text)

                        # Сохраняем данные
                        current_examples.append(
                            [state.global_step, "Случайный", original_text, target_text, generated_text])

            # Добавляем данные текущего шага в общий список
            self.examples_data.extend(current_examples)

            # Создаем новую таблицу из всех накопленных данных
            table = wandb.Table(columns=self.columns, data=self.examples_data)

            # Логируем обновленную таблицу (убран параметр step)
            wandb.log({"examples_table": table})

    def _extract_original_text(self, idx):
        """Извлекает оригинальный текст из датасета."""
        return self.dataset.data.iloc[idx]['original']

    def _extract_target_text(self, idx):
        """Извлекает целевой текст из датасета."""
        return self.dataset.data.iloc[idx]['replaced']

    def _generate_text(self, original_text):
        """Генерирует текст с помощью модели."""
        # Создаем инструкцию для модели
        instruction = (
            "Инструкция: Замените англицизмы в тексте на их русские аналоги.\n\n"
            f"Текст: {original_text}\n\n"
            f"Результат:"
        )

        # Токенизация
        inputs = self.tokenizer(instruction, return_tensors="pt").to(self.model.device)

        # Отключаем вывод warning-ов
        import warnings
        warnings.filterwarnings("ignore")

        # Генерация текста
        with torch.no_grad():
            outputs = self.model.generate(
                input_ids=inputs.input_ids,
                attention_mask=inputs.attention_mask,
                max_new_tokens=256,
                temperature=0.7,
                top_p=0.9,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id,
                # Добавляем параметры для предотвращения повторений
                repetition_penalty=1.2,
                no_repeat_ngram_size=3
            )

        # Декодирование результата
        generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Извлекаем только часть после "Результат:"
        result_parts = generated_text.split("Результат:")
        if len(result_parts) > 1:
            result_text = result_parts[1].strip()

            # Удаляем повторяющиеся фрагменты текста
            sentences = result_text.split('.')
            if sentences:
                # Берем только первое предложение и добавляем точку, если её нет
                clean_result = sentences[0].strip()
                if not clean_result.endswith('.'):
                    clean_result += '.'

                return clean_result

            return result_text
        else:
            return "Ошибка генерации"


class AngliclsmReplacementTrainer:
    def __init__(
            self,
            model_name="ai-forever/rugpt3large_based_on_gpt2",  # Изменено на rugpt3
            dataset_path="train_dataset.csv",
            validation_path="val_dataset.csv",
            validation_fraction=0.1,
            output_dir="/content/assets/train/",
            wandb_project="anglicism-replacement",
            wandb_entity=None,
            lora_r=16,
            lora_alpha=32,
            lora_dropout=0.05,
            batch_size=1,
            num_epochs=3,
            learning_rate=1e-4,
            max_length=256,
            weight_decay=0.01,
            warmup_ratio=0.1,
            save_steps=100,
            eval_steps=1000,
            save_total_limit=5,
            log_examples_every=50,
            num_fixed_examples=3,
            num_random_examples=3,
            device="cuda" if torch.cuda.is_available() else "cpu"
    ):
        self.model_name = model_name
        self.dataset_path = dataset_path
        self.validation_path = validation_path
        self.validation_fraction = validation_fraction
        self.output_dir = output_dir
        self.wandb_project = wandb_project
        self.wandb_entity = wandb_entity
        self.lora_r = lora_r
        self.lora_alpha = lora_alpha
        self.lora_dropout = lora_dropout
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.learning_rate = learning_rate
        self.max_length = max_length
        self.weight_decay = weight_decay
        self.warmup_ratio = warmup_ratio
        self.save_steps = save_steps
        self.eval_steps = eval_steps
        self.save_total_limit = save_total_limit
        self.log_examples_every = log_examples_every
        self.num_fixed_examples = num_fixed_examples
        self.num_random_examples = num_random_examples
        self.device = device

        # Создаем директорию для сохранения модели, если она не существует
        os.makedirs(output_dir, exist_ok=True)

        # Инициализация wandb
        wandb.init(
            project=self.wandb_project,
            entity=self.wandb_entity,
            config={
                "model_name": model_name,
                "lora_r": lora_r,
                "lora_alpha": lora_alpha,
                "lora_dropout": lora_dropout,
                "batch_size": batch_size,
                "num_epochs": num_epochs,
                "learning_rate": learning_rate,
                "max_length": max_length,
                "weight_decay": weight_decay,
                "warmup_ratio": warmup_ratio,
                "save_steps": save_steps,
                "eval_steps": eval_steps,
                "save_total_limit": save_total_limit,
                "validation_fraction": validation_fraction,
                "num_fixed_examples": num_fixed_examples,
                "num_random_examples": num_random_examples,
            }
        )

    def _load_tokenizer_and_model(self):
        """Загрузка токенизатора и модели"""
        print(f"Загрузка модели {self.model_name}...")
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)

        # Добавляем специальные токены для маркировки англицизмов
        special_tokens = {"additional_special_tokens": ["<англицизм>", "</англицизм>"]}
        self.tokenizer.add_special_tokens(special_tokens)

        # Для rugpt3 не требуется дополнительных токенов для форматирования диалогов

        # Загружаем модель
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            torch_dtype=torch.float16,
            device_map="auto"
        )

        # Изменяем размер эмбеддингов для новых токенов
        self.model.resize_token_embeddings(len(self.tokenizer))

        # Подготовка модели для 4-битного обучения и настройка LoRA
        self.model = prepare_model_for_kbit_training(self.model)

        # Настройка конфигурации LoRA для GPT-2 архитектуры
        lora_config = LoraConfig(
            task_type=TaskType.CAUSAL_LM,
            r=self.lora_r,
            lora_alpha=self.lora_alpha,
            lora_dropout=self.lora_dropout,
            bias="none",
            # Целевые модули для GPT-2 архитектуры
            target_modules=["c_attn", "c_proj", "c_fc"]
        )

        # Получаем PEFT модель
        self.model = get_peft_model(self.model, lora_config)
        print(f"Модель загружена и подготовлена для обучения с LoRA")

    def _create_dataset(self, data_path, fraction=1.0):
        """Создание датасета из CSV файла с возможностью выбора доли данных"""

        class AnglicismDataset(Dataset):
            def __init__(self, data_path, tokenizer, max_length, fraction=1.0):
                # Загружаем данные
                full_data = pd.read_csv(data_path)

                # Если нужна только часть данных, берем случайную выборку
                if fraction < 1.0:
                    self.data = full_data.sample(frac=fraction, random_state=42)
                    print(f"Используется {len(self.data)} из {len(full_data)} примеров ({fraction * 100:.1f}%)")
                else:
                    self.data = full_data

                self.tokenizer = tokenizer
                self.max_length = max_length

            def __len__(self):
                return len(self.data)

            def __getitem__(self, idx):
                original = self.data.iloc[idx]['original']
                replaced = self.data.iloc[idx]['replaced']

                # Создаем инструкцию для модели с форматом для GPT
                instruction = (
                    "Инструкция: Замените англицизмы в тексте на их русские аналоги.\n\n"
                    f"Текст: {original}\n\n"
                    f"Результат: {replaced}"
                )

                # Токенизация ввода
                encoded = self.tokenizer(
                    instruction,
                    max_length=self.max_length,
                    padding="max_length",
                    truncation=True,
                    return_tensors="pt"
                )

                # Готовим inputs и labels для обучения
                input_ids = encoded["input_ids"].squeeze(0)
                attention_mask = encoded["attention_mask"].squeeze(0)

                # Для обучения labels должны быть такими же, как input_ids
                labels = input_ids.clone()

                # Устанавливаем -100 для токенов, которые не нужно учитывать при обучении
                # Только для части инструкции и исходного текста
                prompt_end_pos = instruction.find("Результат:") + len("Результат:")
                prompt_tokens = self.tokenizer(
                    instruction[:prompt_end_pos],
                    add_special_tokens=False
                )["input_ids"]
                labels[:len(prompt_tokens)] = -100

                return {
                    "input_ids": input_ids,
                    "attention_mask": attention_mask,
                    "labels": labels
                }

        return AnglicismDataset(data_path, self.tokenizer, self.max_length, fraction=fraction)

    def _prepare_dataset(self):
        """Подготовка тренировочного датасета"""
        print("Загрузка и подготовка тренировочного датасета...")
        self.dataset = self._create_dataset(self.dataset_path, fraction=1.0)
        print(f"Тренировочный датасет подготовлен, количество примеров: {len(self.dataset)}")

    def _prepare_validation_dataset(self):
        """Подготовка валидационного датасета с использованием указанной доли данных"""
        print(
            f"Загрузка и подготовка валидационного датасета (используется {self.validation_fraction * 100:.1f}% данных)...")
        self.validation_dataset = self._create_dataset(
            self.validation_path,
            fraction=self.validation_fraction
        )
        print(f"Валидационный датасет подготовлен, количество примеров: {len(self.validation_dataset)}")

    def train(self):
        """Обучение модели"""
        self._load_tokenizer_and_model()
        self._prepare_dataset()
        self._prepare_validation_dataset()

        # Настройка параметров обучения с новыми опциями
        training_args = TrainingArguments(
            output_dir=self.output_dir,
            overwrite_output_dir=True,
            num_train_epochs=self.num_epochs,
            per_device_train_batch_size=self.batch_size,
            per_device_eval_batch_size=self.batch_size,
            learning_rate=self.learning_rate,
            weight_decay=self.weight_decay,
            warmup_ratio=self.warmup_ratio,
            save_strategy="steps",
            save_steps=self.save_steps,
            save_total_limit=self.save_total_limit,
            eval_strategy="steps",
            eval_steps=self.eval_steps,
            load_best_model_at_end=True,
            metric_for_best_model="eval_loss",
            greater_is_better=False,
            fp16=True,
            report_to="wandb",
            remove_unused_columns=False,
            gradient_accumulation_steps=16,
            optim="adamw_torch",
            logging_steps=10
        )

        # Инициализация тренера с добавлением валидационного датасета
        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=self.dataset,
            eval_dataset=self.validation_dataset,
            data_collator=DataCollatorForLanguageModeling(tokenizer=self.tokenizer, mlm=False)
        )

        # Добавляем собственный колбэк для логирования примеров
        wandb_table_callback = WandbTableCallback(
            trainer=trainer,
            dataset=self.dataset,
            tokenizer=self.tokenizer,
            model=self.model,
            num_fixed_examples=self.num_fixed_examples,
            num_random_examples=self.num_random_examples
        )

        trainer.add_callback(wandb_table_callback)

        # Запуск обучения
        print("Начало обучения...")
        trainer.train()

        # Сохранение модели и токенизатора
        self.model.save_pretrained(self.output_dir)
        self.tokenizer.save_pretrained(self.output_dir)
        print(f"Обучение завершено. Модель сохранена в {self.output_dir}")

        # Завершаем сессию wandb
        wandb.finish()


if __name__ == "__main__":
    # Пример использования класса с новыми параметрами
    trainer = AngliclsmReplacementTrainer(
        model_name="ai-forever/rugpt3large_based_on_gpt2",  # Используем rugpt3 от ai-forever
        validation_fraction=0.05,
        save_steps=150,
        eval_steps=150,
        save_total_limit=5
    )
    trainer.train()

wandb: Currently logged in as: gordeevanatalia04 (gordeevanatalia04-financial-university-under-the-governm) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Загрузка модели ai-forever/rugpt3large_based_on_gpt2...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

...

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
150,3.946900,3.228956
300,1.653700,1.708426
450,1.582100,1.549360
600,1.500600,1.499411
750,1.481700,1.473966


Step,Training Loss,Validation Loss
150,3.946900,3.228956
300,1.653700,1.708426
450,1.582100,1.549360
600,1.500600,1.499411
750,1.481700,1.473966
900,1.463000,1.455134
1050,1.486100,1.445295
1200,1.435000,1.433846
1350,1.426400,1.423861
1500,1.393200,1.421655


Обучение завершено. Модель сохранена в /content/assets/train/


eval/loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/runtime,▇▇▁▆█▂▅▁▆▆▁▇█▆▇▆▇▆▄▁▃▇▂
eval/samples_per_second,▂▂█▃▁▇▄█▃▃█▂▁▃▂▃▂▃▅█▆▂▇
eval/steps_per_second,▂▂█▃▁▇▄█▃▃█▂▁▃▂▃▂▃▅█▆▂▇
train/epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▄▄▄▄▄▄▄▄▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇█████
train/grad_norm,▇▇▆▅█▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,▁▆▇██▇▇▇▇▇▇▆▆▆▆▅▅▅▅▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▁
train/loss,█▇▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/loss,1.38694
eval/runtime,7.5995


In [4]:
from huggingface_hub import notebook_login

# Запустите это и вставьте ваш токен в появившееся поле
notebook_login()

In [ ]:
from huggingface_hub import HfApi
from pathlib import Path

api = HfApi()
repo_id = "nata2627/angl_detection_tokenizer_ruGPT"

# First verify the path
model_path = "/content/assets/train/checkpoint-3465"
if not Path(model_path).is_dir():
    raise ValueError(f"Model directory not found at: {model_path}")

# Then upload
api.upload_folder(
    folder_path=model_path,
    repo_id=repo_id,
    repo_type="model"
)

adapter_model.safetensors:   0%|          | 0.00/655M [00:00<?, ?B/s]

optimizer.pt:   0%|          | 0.00/75.7M [00:00<?, ?B/s]

scaler.pt:   0%|          | 0.00/988 [00:00<?, ?B/s]

Upload 6 LFS files:   0%|          | 0/6 [00:00<?, ?it/s]

rng_state.pth:   0%|          | 0.00/14.2k [00:00<?, ?B/s]

scheduler.pt:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]